In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tqdm.notebook import tqdm
from keras.utils import to_categorical
import gensim
import numpy as np
from keras.callbacks import callbacks
from sentence_transformers import SentenceTransformer
from sklearn.metrics import f1_score

Using TensorFlow backend.
/home/nikamanth/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nikamanth/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nikamanth/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nikamanth/.local/lib/python3.7/site-packages/tenso

In [2]:
def write_result(file_name, preds):
    with open('../results/'+file_name, 'wt') as file:
        for i in preds:
            file.write(i+'\n')
    with open('../results/aravec_result.txt', 'wt') as file:
        for i in preds:
            file.write(i+'\n')
    !python ../NADI-2020_release_1.0/NADI_release/NADI-DID-Scorer.py ../tsv/gold1.txt ../results/aravec_result.txt

def write_result2(file_name, preds):
    with open('../results/'+file_name, 'wt') as file:
        for i in preds:
            file.write(i+'\n')
    with open('../results/aravec_result.txt', 'wt') as file:
        for i in preds:
            file.write(i+'\n')
    !python ../NADI-2020_release_1.0/NADI_release/NADI-DID-Scorer.py ../tsv/gold2.txt ../results/aravec_result.txt

def preprocess_text(train_list, test_list):
    X_train_corrected_tweets = []
    for tweet in tqdm(train_list):
        new_tweet = re.findall( '[^A-Za-z:/_.0-9\\#@,=+\(\)]+' ,tweet)
        new_tweet = " ".join(new_tweet).replace('\xa0','').replace('\u200c','').replace('\U000fe329','').replace('\u2066','').replace('\u2069','').strip()
        X_train_corrected_tweets.append(new_tweet)

    X_dev_corrected_tweets = []
    for tweet in tqdm(test_list):
        new_tweet = re.findall( '[^A-Za-z:/_.0-9\\#@,=+\(\)]+' ,tweet) #[^\x00-\x19\x21-\x7F]+
        new_tweet = " ".join(new_tweet).replace('\xa0','').replace('\u200c','').replace('\U000fe329','').replace('\u2066','').replace('\u2069','').strip()
        X_dev_corrected_tweets.append(new_tweet)
    return X_train_corrected_tweets, X_dev_corrected_tweets

In [3]:
#aravec
t_model = gensim.models.Word2Vec.load(
    '../downloads/aravec/full_uni_cbow_100_twitter/full_uni_cbow_100_twitter.mdl')

# task1

# unbalanced

In [4]:
train_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/train_labeled.tsv',sep='\t')
dev_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/dev_labeled.tsv',sep='\t')

X_train_original,y_train_original = train_df["#2 tweet_content"],train_df["#3 country_label"]
X_dev_original,y_dev_original = dev_df["#2 tweet_content"],dev_df["#3 country_label"]

In [5]:
labels = {}
uni = y_train_original.unique()
for i in range(len(uni)):
    labels[uni[i]] = i

y_train_index = [labels[i] for i in y_train_original]
y_dev_index = [labels[i] for i in y_dev_original]

y_train = to_categorical(y_train_index, num_classes=21)
y_dev = to_categorical(y_dev_index, num_classes=21)

In [6]:
X_noov = []
for sentence in X_train_original:
    sentence = sentence.split(' ')
    new_sentence = []
    for word in sentence:
        if word in t_model.wv.vocab:
            new_sentence.append(word)
    X_noov.append(new_sentence)
X_noov_dev = []
for sentence in X_dev_original:
    sentence = sentence.split(' ')
    new_sentence = []
    for word in sentence:
        if word in t_model.wv.vocab:
            new_sentence.append(word)
    X_noov_dev.append(new_sentence)
    
X_train = []
counter = 0
for one_vec in X_noov:
    if one_vec == []:
        counter += 1
        one_vec = ['ومايشوف']
    word_vector = t_model.wv[ one_vec ]
    word_vector = np.sum(word_vector,axis=0)
#     word_vector = np.pad(word_vector,pad_width=((0,100-word_vector.shape[0]),(0,0)))
    X_train.append(word_vector)
print(counter)
X_dev = []
counter = 0
for one_vec in X_noov_dev:
    if one_vec == []:
        counter += 1
        one_vec = ['ومايشوف']
    word_vector = t_model.wv[ one_vec ]
    word_vector = np.sum(word_vector,axis=0)
#     word_vector = np.pad(word_vector,pad_width=((0,100-word_vector.shape[0]),(0,0)))
    X_dev.append(word_vector)

X_train = np.array(X_train)
X_dev = np.array(X_dev)

292


In [7]:
model = Sequential()
model.add(Dense(512, activation='relu'))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train,y_train,epochs=50,validation_data=(X_dev,y_dev),
                   callbacks=[callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
                                                      patience=10, verbose=0, mode='auto', 
                                                      baseline=None, restore_best_weights=True)])
prediction = model.predict_classes(X_dev)


Train on 21000 samples, validate on 4957 samples
Epoch 1/50
21000/21000 [==============================] - 1s 50us/step - loss: 3.4306 - accuracy: 0.2510 - val_loss: 2.7258 - val_accuracy: 0.2842
Epoch 2/50
21000/21000 [==============================] - 1s 53us/step - loss: 2.3987 - accuracy: 0.3253 - val_loss: 2.5807 - val_accuracy: 0.3036
Epoch 3/50
21000/21000 [==============================] - 1s 54us/step - loss: 2.2184 - accuracy: 0.3585 - val_loss: 2.5499 - val_accuracy: 0.3123
Epoch 4/50
21000/21000 [==============================] - 1s 51us/step - loss: 2.1033 - accuracy: 0.3810 - val_loss: 2.6161 - val_accuracy: 0.3060
Epoch 5/50
21000/21000 [==============================] - 1s 47us/step - loss: 2.0171 - accuracy: 0.4000 - val_loss: 2.6449 - val_accuracy: 0.3024
Epoch 6/50
21000/21000 [==============================] - 1s 49us/step - loss: 1.9444 - accuracy: 0.4190 - val_loss: 2.7304 - val_accuracy: 0.3076
Epoch 7/50
21000/21000 [==============================] - 1s 49us/st

In [8]:
reverse_label_map = {value : key for (key, value) in labels.items()}
print(reverse_label_map)
pred = model.predict_classes(X_dev)
pred = [reverse_label_map[i] for i in pred]
# print(pred)
len(pred)
write_result('aravec_task1_unbal.txt',pred)

{0: 'Iraq', 1: 'Egypt', 2: 'Morocco', 3: 'Libya', 4: 'United_Arab_Emirates', 5: 'Mauritania', 6: 'Saudi_Arabia', 7: 'Bahrain', 8: 'Syria', 9: 'Djibouti', 10: 'Lebanon', 11: 'Oman', 12: 'Palestine', 13: 'Algeria', 14: 'Somalia', 15: 'Jordan', 16: 'Tunisia', 17: 'Kuwait', 18: 'Yemen', 19: 'Sudan', 20: 'Qatar'}

OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 13.21 %
MACRO AVERAGE RECALL SCORE: 12.37 %
MACRO AVERAGE F1 SCORE: 11.59 %
OVERALL ACCURACY: 31.23 %



# balanced

In [9]:
train_df = pd.read_csv('../tsv/oversample_train.tsv',sep='\t')
dev_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/dev_labeled.tsv',sep='\t')

X_train_original,y_train_original = train_df["#2 tweet_content"],train_df["#3 country_label"]
X_dev_original,y_dev_original = dev_df["#2 tweet_content"],dev_df["#3 country_label"]

In [10]:
labels = {}
uni = y_train_original.unique()
for i in range(len(uni)):
    labels[uni[i]] = i

y_train_index = [labels[i] for i in y_train_original]
y_dev_index = [labels[i] for i in y_dev_original]

y_train = to_categorical(y_train_index, num_classes=21)
y_dev = to_categorical(y_dev_index, num_classes=21)

In [11]:
X_noov = []
for sentence in X_train_original:
    sentence = sentence.split(' ')
    new_sentence = []
    for word in sentence:
        if word in t_model.wv.vocab:
            new_sentence.append(word)
    X_noov.append(new_sentence)
X_noov_dev = []
for sentence in X_dev_original:
    sentence = sentence.split(' ')
    new_sentence = []
    for word in sentence:
        if word in t_model.wv.vocab:
            new_sentence.append(word)
    X_noov_dev.append(new_sentence)
    
X_train = []
counter = 0
for one_vec in X_noov:
    if one_vec == []:
        counter += 1
        one_vec = ['ومايشوف']
    word_vector = t_model.wv[ one_vec ]
    word_vector = np.sum(word_vector,axis=0)
#     word_vector = np.pad(word_vector,pad_width=((0,100-word_vector.shape[0]),(0,0)))
    X_train.append(word_vector)
print(counter)
X_dev = []
counter = 0
for one_vec in X_noov_dev:
    if one_vec == []:
        counter += 1
        one_vec = ['ومايشوف']
    word_vector = t_model.wv[ one_vec ]
    word_vector = np.sum(word_vector,axis=0)
#     word_vector = np.pad(word_vector,pad_width=((0,100-word_vector.shape[0]),(0,0)))
    X_dev.append(word_vector)

X_train = np.array(X_train)
X_dev = np.array(X_dev)

1309


In [12]:
model = Sequential()
model.add(Dense(512, activation='relu'))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train,y_train,epochs=50,validation_data=(X_dev,y_dev),
                   callbacks=[callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
                                                      patience=10, verbose=0, mode='auto', 
                                                      baseline=None, restore_best_weights=True)])
prediction = model.predict_classes(X_dev)

Train on 93933 samples, validate on 4957 samples
Epoch 1/50
93933/93933 [==============================] - 4s 40us/step - loss: 2.3750 - accuracy: 0.3803 - val_loss: 3.2318 - val_accuracy: 0.1650
Epoch 2/50
93933/93933 [==============================] - 4s 40us/step - loss: 1.4616 - accuracy: 0.5844 - val_loss: 3.5083 - val_accuracy: 0.1832
Epoch 3/50
93933/93933 [==============================] - 4s 41us/step - loss: 1.1686 - accuracy: 0.6677 - val_loss: 4.0113 - val_accuracy: 0.2021
Epoch 4/50
93933/93933 [==============================] - 4s 42us/step - loss: 1.0072 - accuracy: 0.7160 - val_loss: 4.4454 - val_accuracy: 0.2011
Epoch 5/50
93933/93933 [==============================] - 4s 42us/step - loss: 0.8847 - accuracy: 0.7525 - val_loss: 4.9604 - val_accuracy: 0.2033
Epoch 6/50
93933/93933 [==============================] - 4s 41us/step - loss: 0.8154 - accuracy: 0.7755 - val_loss: 5.4539 - val_accuracy: 0.2140
Epoch 7/50
93933/93933 [==============================] - 4s 43us/ste

In [13]:
reverse_label_map = {value : key for (key, value) in labels.items()}
print(reverse_label_map)
pred = model.predict_classes(X_dev)
pred = [reverse_label_map[i] for i in pred]
# print(pred)
len(pred)
write_result('aravec_task1_bal.txt',pred)

{0: 'Lebanon', 1: 'Sudan', 2: 'Djibouti', 3: 'Saudi_Arabia', 4: 'Oman', 5: 'Yemen', 6: 'Bahrain', 7: 'Mauritania', 8: 'Kuwait', 9: 'Iraq', 10: 'Palestine', 11: 'Jordan', 12: 'Somalia', 13: 'Tunisia', 14: 'Morocco', 15: 'Syria', 16: 'United_Arab_Emirates', 17: 'Algeria', 18: 'Egypt', 19: 'Qatar', 20: 'Libya'}

OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 12.82 %
MACRO AVERAGE RECALL SCORE: 12.02 %
MACRO AVERAGE F1 SCORE: 10.47 %
OVERALL ACCURACY: 16.50 %



# task2

In [76]:
import random
from tqdm import tqdm_notebook
train_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/train_labeled.tsv','\t')
g = train_df['#4 province_label'].value_counts()[0]
for i in tqdm_notebook(train_df['#4 province_label'].unique()):
    filter_df = train_df[train_df['#4 province_label'] == i]
    count = len(filter_df)
    while (count<=g):
        count+=1
        idx = random.randint(0,len(filter_df)-1)
        train_df = pd.concat([train_df,filter_df[idx:idx+1]])
train_df['#4 province_label'].value_counts()
train_df.to_csv('../tsv/oversample_train2.tsv')

/home/nikamanth/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


# unbalanced

In [77]:
train_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/train_labeled.tsv',sep='\t')
dev_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/dev_labeled.tsv',sep='\t')

X_train_original,y_train_original = train_df["#2 tweet_content"],train_df["#4 province_label"]
X_dev_original,y_dev_original = dev_df["#2 tweet_content"],dev_df["#4 province_label"]

In [78]:
labels = {}
uni = y_train_original.unique()
for i in range(len(uni)):
    labels[uni[i]] = i

y_train_index = [labels[i] for i in y_train_original]
y_dev_index = [labels[i] for i in y_dev_original]

y_train = to_categorical(y_train_index, num_classes=100)
y_dev = to_categorical(y_dev_index, num_classes=100)

In [79]:
X_noov = []
for sentence in X_train_original:
    sentence = sentence.split(' ')
    new_sentence = []
    for word in sentence:
        if word in t_model.wv.vocab:
            new_sentence.append(word)
    X_noov.append(new_sentence)
X_noov_dev = []
for sentence in X_dev_original:
    sentence = sentence.split(' ')
    new_sentence = []
    for word in sentence:
        if word in t_model.wv.vocab:
            new_sentence.append(word)
    X_noov_dev.append(new_sentence)
    
X_train = []
counter = 0
for one_vec in X_noov:
    if one_vec == []:
        counter += 1
        one_vec = ['ومايشوف']
    word_vector = t_model.wv[ one_vec ]
    word_vector = np.sum(word_vector,axis=0)
#     word_vector = np.pad(word_vector,pad_width=((0,100-word_vector.shape[0]),(0,0)))
    X_train.append(word_vector)
print(counter)
X_dev = []
counter = 0
for one_vec in X_noov_dev:
    if one_vec == []:
        counter += 1
        one_vec = ['ومايشوف']
    word_vector = t_model.wv[ one_vec ]
    word_vector = np.sum(word_vector,axis=0)
#     word_vector = np.pad(word_vector,pad_width=((0,100-word_vector.shape[0]),(0,0)))
    X_dev.append(word_vector)

X_train = np.array(X_train)
X_dev = np.array(X_dev)

292


In [80]:
model = Sequential()
model.add(Dense(512, activation='relu'))
model.add(Dense(100, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train,y_train,epochs=50,validation_data=(X_dev,y_dev),
                   callbacks=[callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
                                                      patience=10, verbose=0, mode='auto', 
                                                      baseline=None, restore_best_weights=True)])
prediction = model.predict_classes(X_dev)

Train on 21000 samples, validate on 4957 samples
Epoch 1/50
21000/21000 [==============================] - 1s 69us/step - loss: 5.4428 - accuracy: 0.0354 - val_loss: 4.7153 - val_accuracy: 0.0242
Epoch 2/50
21000/21000 [==============================] - 1s 65us/step - loss: 4.1859 - accuracy: 0.0777 - val_loss: 4.7348 - val_accuracy: 0.0270
Epoch 3/50
21000/21000 [==============================] - 1s 64us/step - loss: 3.9014 - accuracy: 0.1180 - val_loss: 4.7843 - val_accuracy: 0.0309
Epoch 4/50
21000/21000 [==============================] - 1s 64us/step - loss: 3.6782 - accuracy: 0.1530 - val_loss: 4.9268 - val_accuracy: 0.0343
Epoch 5/50
21000/21000 [==============================] - 1s 64us/step - loss: 3.4886 - accuracy: 0.1867 - val_loss: 5.1040 - val_accuracy: 0.0375
Epoch 6/50
21000/21000 [==============================] - 1s 68us/step - loss: 3.3198 - accuracy: 0.2178 - val_loss: 5.2999 - val_accuracy: 0.0299
Epoch 7/50
21000/21000 [==============================] - 1s 64us/ste

In [81]:
reverse_label_map = {value : key for (key, value) in labels.items()}
print(reverse_label_map)
pred = model.predict_classes(X_dev)
pred = [reverse_label_map[i] for i in pred]
# print(pred)
len(pred)
write_result2('aravec_task2_unbal.txt',pred)

{0: 'iq_Al-Anbar', 1: 'eg_Alexandria', 2: 'iq_Maysan', 3: 'ma_Oriental', 4: 'ly_Al-Jabal-al-Akhdar', 5: 'ae_Fujairah', 6: 'eg_Ismailia', 7: 'iq_Baghdad', 8: 'eg_Dakahlia', 9: 'mr_Nouakchott', 10: 'eg_Qena', 11: 'ma_Marrakech-Tensift-Al-Haouz', 12: 'sa_Tabuk', 13: 'eg_Asyut', 14: 'iq_Karbala', 15: 'bh_Capital', 16: 'sy_Damascus-City', 17: 'sa_Najran', 18: 'dj_Djibouti', 19: 'lb_Akkar', 20: 'om_Musandam', 21: 'ps_Gaza-Strip', 22: 'dz_Oran', 23: 'so_Banaadir', 24: 'sy_As-Suwayda', 25: 'eg_Faiyum', 26: 'jo_Aqaba', 27: 'eg_Cairo', 28: 'lb_North-Lebanon', 29: 'eg_Port-Said', 30: 'eg_Monufia', 31: 'tn_Sousse', 32: 'eg_Beheira', 33: 'sa_Ash-Sharqiyah', 34: 'eg_Gharbia', 35: 'ae_Ras-Al-Khaymah', 36: 'eg_Minya', 37: 'om_Al-Batnah', 38: 'kw_Jahra', 39: 'dz_Jijel', 40: 'dz_Béchar', 41: 'eg_Sohag', 42: 'sy_Hims', 43: 'iq_An-Najaf', 44: 'lb_South-Lebanon', 45: 'eg_Aswan', 46: 'ye_Ibb', 47: 'iq_Dihok', 48: 'sy_Aleppo', 49: 'iq_Al-Muthannia', 50: 'dz_Bordj-Bou-Arreridj\u200e', 51: 'ly_Tripoli', 52: 'y

# balanced

In [85]:
train_df = pd.read_csv('../tsv/oversample_train2.tsv')
dev_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/dev_labeled.tsv',sep='\t')

X_train_original,y_train_original = train_df["#2 tweet_content"],train_df["#4 province_label"]
X_dev_original,y_dev_original = dev_df["#2 tweet_content"],dev_df["#4 province_label"]

In [86]:
labels = {}
uni = y_train_original.unique()
for i in range(len(uni)):
    labels[uni[i]] = i

y_train_index = [labels[i] for i in y_train_original]
y_dev_index = [labels[i] for i in y_dev_original]

y_train = to_categorical(y_train_index, num_classes=100)
y_dev = to_categorical(y_dev_index, num_classes=100)

In [87]:
X_noov = []
for sentence in X_train_original:
    sentence = sentence.split(' ')
    new_sentence = []
    for word in sentence:
        if word in t_model.wv.vocab:
            new_sentence.append(word)
    X_noov.append(new_sentence)
X_noov_dev = []
for sentence in X_dev_original:
    sentence = sentence.split(' ')
    new_sentence = []
    for word in sentence:
        if word in t_model.wv.vocab:
            new_sentence.append(word)
    X_noov_dev.append(new_sentence)
    
X_train = []
counter = 0
for one_vec in X_noov:
    if one_vec == []:
        counter += 1
        one_vec = ['ومايشوف']
    word_vector = t_model.wv[ one_vec ]
    word_vector = np.sum(word_vector,axis=0)
#     word_vector = np.pad(word_vector,pad_width=((0,100-word_vector.shape[0]),(0,0)))
    X_train.append(word_vector)
print(counter)
X_dev = []
counter = 0
for one_vec in X_noov_dev:
    if one_vec == []:
        counter += 1
        one_vec = ['ومايشوف']
    word_vector = t_model.wv[ one_vec ]
    word_vector = np.sum(word_vector,axis=0)
#     word_vector = np.pad(word_vector,pad_width=((0,100-word_vector.shape[0]),(0,0)))
    X_dev.append(word_vector)

X_train = np.array(X_train)
X_dev = np.array(X_dev)

538


In [88]:
model = Sequential()
model.add(Dense(512, activation='relu'))
model.add(Dense(100, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train,y_train,epochs=50,validation_data=(X_dev,y_dev),
                   callbacks=[callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
                                                      patience=10, verbose=0, mode='auto', 
                                                      baseline=None, restore_best_weights=True)])
prediction = model.predict_classes(X_dev)

Train on 39600 samples, validate on 4957 samples
Epoch 1/50
39600/39600 [==============================] - 3s 71us/step - loss: 4.7126 - accuracy: 0.0879 - val_loss: 4.8419 - val_accuracy: 0.0268
Epoch 2/50
39600/39600 [==============================] - 3s 64us/step - loss: 3.5838 - accuracy: 0.2004 - val_loss: 5.1013 - val_accuracy: 0.0305
Epoch 3/50
39600/39600 [==============================] - 3s 85us/step - loss: 3.1615 - accuracy: 0.2847 - val_loss: 5.4247 - val_accuracy: 0.0351
Epoch 4/50
39600/39600 [==============================] - 3s 71us/step - loss: 2.8233 - accuracy: 0.3561 - val_loss: 5.9098 - val_accuracy: 0.0367
Epoch 5/50
39600/39600 [==============================] - 3s 64us/step - loss: 2.5386 - accuracy: 0.4152 - val_loss: 6.3618 - val_accuracy: 0.0343
Epoch 6/50
39600/39600 [==============================] - 2s 62us/step - loss: 2.2948 - accuracy: 0.4685 - val_loss: 7.0649 - val_accuracy: 0.0347
Epoch 7/50
39600/39600 [==============================] - 3s 63us/ste

In [89]:
reverse_label_map = {value : key for (key, value) in labels.items()}
print(reverse_label_map)
pred = model.predict_classes(X_dev)
pred = [reverse_label_map[i] for i in pred]
# print(pred)
len(pred)
write_result2('aravec_task2_unbal.txt',pred)

{0: 'iq_Al-Anbar', 1: 'eg_Alexandria', 2: 'iq_Maysan', 3: 'ma_Oriental', 4: 'ly_Al-Jabal-al-Akhdar', 5: 'ae_Fujairah', 6: 'eg_Ismailia', 7: 'iq_Baghdad', 8: 'eg_Dakahlia', 9: 'mr_Nouakchott', 10: 'eg_Qena', 11: 'ma_Marrakech-Tensift-Al-Haouz', 12: 'sa_Tabuk', 13: 'eg_Asyut', 14: 'iq_Karbala', 15: 'bh_Capital', 16: 'sy_Damascus-City', 17: 'sa_Najran', 18: 'dj_Djibouti', 19: 'lb_Akkar', 20: 'om_Musandam', 21: 'ps_Gaza-Strip', 22: 'dz_Oran', 23: 'so_Banaadir', 24: 'sy_As-Suwayda', 25: 'eg_Faiyum', 26: 'jo_Aqaba', 27: 'eg_Cairo', 28: 'lb_North-Lebanon', 29: 'eg_Port-Said', 30: 'eg_Monufia', 31: 'tn_Sousse', 32: 'eg_Beheira', 33: 'sa_Ash-Sharqiyah', 34: 'eg_Gharbia', 35: 'ae_Ras-Al-Khaymah', 36: 'eg_Minya', 37: 'om_Al-Batnah', 38: 'kw_Jahra', 39: 'dz_Jijel', 40: 'dz_Béchar', 41: 'eg_Sohag', 42: 'sy_Hims', 43: 'iq_An-Najaf', 44: 'lb_South-Lebanon', 45: 'eg_Aswan', 46: 'ye_Ibb', 47: 'iq_Dihok', 48: 'sy_Aleppo', 49: 'iq_Al-Muthannia', 50: 'dz_Bordj-Bou-Arreridj\u200e', 51: 'ly_Tripoli', 52: 'y